In [60]:
import torch
from datasets import load_dataset, load_from_disk
import math

import IPython
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import collections
import json
import math
import os
import re
import time

import ahocorasick
import networkx as nx
import openai
import pandas as pd
import pdfplumber
import torch
import transformers
from fuzzywuzzy import fuzz
from matplotlib import pyplot as plt

dataDir = "../data/"
dataName = "Deep Learning.pdf"
import codecs
import copy
import random

import numpy as np
from tqdm.notebook import tqdm
from transformers import AutoModelForTokenClassification, AutoTokenizer, pipeline

# openai.api_base = "https://api.chatanywhere.com.cn/"
openai.api_base = "https://api.chatanywhere.cn/"
openai.api_key = "sk-LzwgVgu5xvNPpwoqCdeeVcAt7Tu7ZoZICXzzkheldIbXA60h"

# 1. 获取模型

In [33]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-large-cased', cache_dir="../../../BERT/large")
model = BertModel.from_pretrained("bert-large-cased", cache_dir="../../../BERT/large")

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

G:\Crime\Anaconda3\envs\pytorch\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in G:\Crime\BERT\large\models--bert-large-cased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

# 2. 获取数据

In [87]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [15]:
with pdfplumber.open(dataDir + dataName) as f:
    # 目录架构生成
    c, p, n = [], [], []
    for i in range(7):
        page = f.pages[i]
        text = page.extract_text()
        text_split = text.split("\n")
        for i in text_split:
            if bool(re.match("[0-9]+\.[0-9]+", i.split(" ")[0])):
                c.append(i.split(" ")[0])
                p.append(int(i.split(" ")[-1]) + 15)
            if bool(re.match("[0-9]+", i.split(" ")[0])):
                for j in i.split(" "):
                    if bool(re.match("[A-Za-z]+", j)):
                        n.append((i.split(" ")[0], j))
                        
p_range = list(zip(p, p[1:]))
p_range.append((735, 800))
c_p_range = list(zip(c, p_range))
index_dict = collections.defaultdict(list)
for k, v in c_p_range:
    index_dict[k.split(".")[0]].append((k, v))

with pdfplumber.open(dataDir + dataName) as f:
    content_dict = collections.defaultdict(list)

    for k, v in tqdm(index_dict.items(), total=len(index_dict)):
        for i in v:
            page_range = i[-1]
            if page_range[0] == page_range[1]:
                page_range = (page_range[0], page_range[1] + 1)
            for j in range(int(page_range[0]) - 1, int(page_range[1]) - 1):
                page = f.pages[j]

                text = page.extract_text().replace("\n", " ")

                content_dict[i[0]].append(text)

  0%|          | 0/20 [00:00<?, ?it/s]

In [143]:
text = content_dict['1.1'][1]
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)

In [150]:
encoded_input['input_ids'][0,0]

tensor(101)

In [145]:
tokenizer.batch_decode(encoded_input['input_ids'])

['[CLS] CHAPTER 1. INTRODUCTION Deep learning Example : Shallow Example : Example : Example : autoencoders Logistic Knowledge MLPs regression bases Representation learning Machine learning AI Figure 1. 4 : A Venn diagram showing how deep learning is a kind of representation learning, which is in turn a kind of machine learning, which is used for many but not all approaches to AI. Each section of the Venn diagram includes an example of an AI technology. 9 [SEP]']

In [58]:
test = tokenizer("you'd", return_tensors='pt')
test

{'input_ids': tensor([[ 101, 1128,  112,  173,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}

In [59]:
tokenizer.batch_decode(test['input_ids'])

["[CLS] you'd [SEP]"]

In [38]:
output

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.3712, -0.6087,  0.6811,  ..., -0.1764, -0.2981,  0.1912],
         [ 0.2954,  0.1486, -0.4758,  ..., -1.0722,  0.7647, -0.2719],
         [-0.0229,  0.2538,  0.4917,  ...,  0.1088, -0.3639,  0.7795],
         ...,
         [ 0.0234,  0.4296,  0.1787,  ...,  0.2195,  0.0112,  0.0642],
         [ 0.9951, -0.5820,  0.9646,  ..., -0.1429, -0.2179, -0.2663],
         [ 1.2194, -1.3121,  0.2408,  ..., -0.6321, -0.3233,  0.8943]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[ 0.9641,  0.9999,  0.9999,  ..., -1.0000,  0.5894,  0.9971]],
       grad_fn=<TanhBackward0>), hidden_states=None, past_key_values=None, attentions=None, cross_attentions=None)

In [106]:
class Chat:
    def __init__(self, conversation_list=[]):
        self.conversation_list = conversation_list
        self.costs_list = []

    def show_conversation(self, msg_list):
        for msg in msg_list[-2:]:
            if msg["role"] == "user":
                pass
            else:
                message = msg["content"]
                pass
                # print(f"\U0001f47D: {message}\n")

    def ask(self, prompt):
        self.conversation_list.append({"role": "user", "content": prompt})
        openai.api_key = "sk-LzwgVgu5xvNPpwoqCdeeVcAt7Tu7ZoZICXzzkheldIbXA60h"
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-16k", messages=self.conversation_list
        )
        answer = response.choices[0].message["content"]

        self.conversation_list.append({"role": "assistant", "content": answer})
        self.show_conversation(self.conversation_list)

        # cost = total_counts(response)
        # self.costs_list.append(cost)
        return answer

In [110]:
NER_prompt = f"""
角色：
你是一个深度学习领域的实体标注专员

任务：
给定字符串，请找出全部深度学习领域的实体

步骤：
请以以下步骤执行：
1. 找出句子中的所有深度学习领域的实体
2. 依次检查实体是否属于深度学习领域
3. 将属于深度学习领域的实体返回
4. 若没有深度学习领域的实体，则返回()

格式：
请以以下格式返回：
('entity1', 'entity2', ...)

举例如下：
input: An illustration of how the gradient descent algorithm uses the derivatives of a function can be used to follow the function downhill to a minimum.
output: (gradient descent algorithm)

input: an encoder or reader or input RNN processes the input sequence. The encoder emits the context C, usually as a simple function of its final hidden state.
output: (encoder, RNN, hidden state)

input: There is no constraint that the encoder must have the same size of hidden layer as the decoder
output: (hidden layer, decoder)

input: Computer vision has traditionally been one of the most active research areas for deep learning applications, because vision is a task that is effortless for humans and many animals but challenging for computers (Ballard et al., 1983)
output: (Computer vision, deep learning)

input: Dataset augmentation may be seen as a way of preprocessing the training set only.
output: (Dataset augmentation)

input: CHAPTER 1. INTRODUCTION of the flowchart of the computations needed to compute the representation of each concept may be much deeper than the graph of the concepts themselves.
output: ()

注意事项：
1. 请严格遵循字符串中的原本表述
2. 除返回结果外，不要返回任何其他内容
"""

In [116]:
check_prompt = f"""
任务：
请检查所给实体是否属于深度学习领域

格式：
请以以下格式返回：
如果该实体是深度学习领域的实体，返回True，否则返回False

举例如下：
input: deep learning
output: True

input: AI system
output: True

input: image
output: False

input: Image Net
output: True

input: face
output: False

注意事项：
1. 请仅返回True或者False，不要返回任何其他内容
"""

In [176]:
total = 0
for k, v in content_dict.items():
    if k in ['4.2', '10.1', '15.1', '20.1']:
        if total >= 1500:
            break
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)
        docs = text_splitter.split_text(' '.join(i for i in v))
        # 将句子输入给大模型提取命名实体
        for index, content in enumerate(docs):
            total += 1
            if total == 1500:
                break
            if index % 5 == 0:
                conversation_list = [
                    {
                        "role": "system",
                        "content": NER_prompt,
                    }
                ]
                bot_ner = Chat(conversation_list)
            answer_ner = bot_ner.ask("input: "+content)
            entity_list_temp = re.sub("\(|\)|", "", answer_ner).split(", ")
            conversation_list = [
                    {
                        "role": "system",
                        "content": check_prompt,
                    }
                ]
            bot_check = Chat(conversation_list)
            entity_list = []
            for e in entity_list_temp:
                answer_check = bot_check.ask("input: "+e)
                if answer_check == 'True':
                    entity_list.append(e)
            # 将句子进行tokenizer
            content_token = tokenizer(content, return_tensors='pt')['input_ids']
            # 将实体进行tokenizer
            entity_token = []
            for e in entity_list:
                entity_token.append(tokenizer(e, return_tensors='pt')['input_ids'])
            # 找到对应位置
            label = [0]*content_token.shape[1]
            for e in entity_token:
                e_temp = []
                for i in range(1, e.shape[1]-1):
                    e_temp.append(e[0,i])
                temp = e_temp.copy()
                i, j = 0, 0
                while j < content_token.shape[1]:
                    if content_token[0,j]!=temp[0]:
                        temp = e_temp.copy()
                        j += 1
                        i = j
                    else:
                        temp.pop(0)
                        j += 1
                    if not temp:
                        temp = e_temp.copy()
                        while i < j:
                            label[i] = 1
                            i += 1

            df = pd.DataFrame(
                    [[tokenizer.batch_decode(content_token), label]],
                    columns=["text", "label"],
                )
            df.to_csv(
                os.path.join(dataDir + "/relations", f"sample.csv"),
                mode="a",
                header=not os.path.exists(
                    os.path.join(dataDir + "/relations", f"sample.csv")
                ),
                index=False,
            )
            # 返回原句和标签

KeyboardInterrupt: 

In [161]:
df = pd.read_csv(
    dataDir + "relations/" + "sample.csv",
    index_col=False)

In [165]:
df['text'][0]

"['[CLS] CHAPTER 1. INTRODUCTION of the flowchart of the computations needed to compute the representation of each concept may be much deeper than the graph of the concepts themselves. This is because the system ’ s understanding of the simpler concepts can be refined given information about the more complex concepts. For example, an AI system observing an image of a face with one eye in shadow may initially only see one eye. After detecting that a face is present, it can then infer that a second eye is [SEP]']"

In [164]:
df['label'][0]

'[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]'